<a href="https://colab.research.google.com/github/PARULCHUTANIPC/parul/blob/p1/Dogs%20Vs%20Cats%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## **PREPARING THE VGG16 MODEL**

**Importing the model**

In [0]:
vgg16 = keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 8s 0us/step


In [0]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

**Removing the output layer**

In [0]:
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# show a summary of the base model
print("[INFO] summary for base model...")
print(baseModel.summary())

**Adding the output layer according to dataset**

In [0]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
'''
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
'''
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

**Compiling the Model**

In [0]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = tf.keras.optimizers.Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

## **PREPARING THE DATASET**

In [8]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/Colab Notebooks/DATASETS/DOGS VS CATS/dogs-vs-cats.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [9]:
from zipfile import ZipFile
train_images = "/content/train.zip"
with ZipFile(train_images,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [10]:
from zipfile import ZipFile
test_images = "/content/test1.zip"
with ZipFile(test_images,'r') as zip:
  zip.extractall()
  print("Done")

Done


**Defining the parameters**

In [0]:
image_width, image_height = 224,224
training_data = train_images
test_data = test_images
no_of_training_samples = 1000
epochs = 10
batch_size =20

**Checking the image format - Channels-width-hight or  width-height-channels**

In [0]:
if K.image_data_format() == 'channels_first':
  input_shape = (3, image_width, image_height)
else:
  input_shape = (image_width, image_height, 3)

**Setting the parameters of the newly generated images**

In [0]:
training_data_generation = ImageDataGenerator(rotation_range=90, shear_range=0.2, horizontal_flip=True, vertical_flip=True, rescale=1/255, zoom_range=0.4)

In [0]:
test_data_generation = ImageDataGenerator(rescale=1/255)


**Generating the images using image data generator**

In [21]:
training_generator = training_data_generation.flow_from_directory(
    "training_data",
    target_size = (224,224),
    color_mode = 'rgb',
    batch_size = batch_size,
    class_mode = 'binary'

)

FileNotFoundError: ignored

## **TRAINING THE MODEL**

In [0]:
model.fit_generator(
    training_generator,
    steps_per_epoch = no_of_training_samples,
    epochs = epochs

)

**Saving the weights & model**

In [0]:
model.save_weights("dogs-cats-try1.h5")

**Selecting 1 image to verify the working of model**

In [0]:
image_pred = image.load_img("       ",target size = (224,224))
image_pred = image.img_to_array(image_pred)
image_pred = np.expand_dims(image_pred, axis = 0)

**Testing the single uploaded image & Verify the result**

In [0]:
result = model.predict(image_pred)
print(image_pred)
if result[0][0] ==1:
  prediction = "This is a Dog"
else:
  prediction = "This is a Cat"
print(prediction)